In [1]:
import torch_geometric.nn as pyg_nn
import torch.nn as nn 
from torch_geometric.datasets import QM9
from torch_geometric.data import DataLoader
from torch_geometric.nn import global_max_pool, Set2Set
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
from rdkit import Chem

In [3]:
from collections import deque

In [11]:
mol =Chem.MolFromSmiles("O=C(CN)N")


In [12]:
Chem.rdmolops.GetAdjacencyMatrix(mol)

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]], dtype=int32)

In [4]:
mem = deque(maxlen=10)

In [5]:
mem.append("Hei")

In [6]:
mem

deque(['Hei'], maxlen=10)

In [7]:
mem.append("Hei2")

In [8]:
mem

deque(['Hei', 'Hei2'], maxlen=10)

In [9]:
mem.appendleft("Hei3")
mem

deque(['Hei3', 'Hei', 'Hei2'], maxlen=10)

In [10]:
mem.appendleft("Hei4")
mem

deque(['Hei4', 'Hei3', 'Hei', 'Hei2'], maxlen=10)

In [3]:
table = Chem.GetPeriodicTable()

In [8]:
atoms = ["C", "O", "H"]

In [12]:
def atom_valence(atoms):
    table = Chem.GetPeriodicTable()
    return [max(list(table.GetValenceList(atom))) for atom in atoms]

In [14]:
valence = atom_valence(atoms)


In [15]:
atom_valences = { atom_type: atom_valence([atom_type])[0] for atom_type in atoms}

In [17]:
atom_valences.values()


dict_values([4, 2, 1])

In [19]:
import itertools

In [21]:
list(itertools.combinations(atoms, 2))

[('C', 'O'), ('C', 'H'), ('O', 'H')]

In [ ]:

atoms_with_free_valence = {}
for i in range(1, max(atom_valences.values())):
    # Only atoms that allow us to replace at least one H with a new bond are
    # enumerated here.
    atoms_with_free_valence[i] = [
        atom.GetIdx() for atom in mol.GetAtoms() if atom.GetNumImplicitHs() >= i
    ]
valid_ac

In [10]:
num = torch.tensor((3))

num.sqrt()

num.exp()

tensor(20.0855)

In [9]:
torch.exp(torch.tensor((3)))

tensor(20.0855)

In [11]:
import collections

In [13]:
class Result(collections.namedtuple("Result", ["state", "reward", "terminated"])): 
    """Name"""

In [17]:
Point = collections.namedtuple("Res", ["x", "y"])
p = Point(1,1)
p

Res(x=1, y=1)

In [15]:
result = Result("state", 1, 0)
result

Result(state='state', reward=1, terminated=0)

In [13]:
a = torch.randn((10,10))
mask = torch.ones((10,10))
upper = torch.triu(mask)

In [16]:
f = lambda x: x * 2

upper.apply_(f)

tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

In [37]:
a = torch.tensor(([1,2,3], [4,5,6]))
b = torch.tensor([9,10,11])
a.shape,b.shape, b.unsqueeze(0).shape


(torch.Size([2, 3]), torch.Size([3]), torch.Size([1, 3]))

In [36]:
torch.cat((a, b.unsqueeze(0)), dim=0)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 9, 10, 11]])

In [34]:
torch.cat((a,b, dim=-1)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 2 but got size 3 for tensor number 1 in the list.

In [39]:
root = r"C:\Users\tnf\Documents\NTNU\2024\WaterlooInternship\final_project\data\qm9"
data = QM9


In [40]:
data = data(root=root)

c:\Users\tnf\Documents\NTNU\2024\WaterlooInternship\final_project\env\lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if

In [41]:
train_loader = DataLoader(data, batch_size=4)


c:\Users\tnf\Documents\NTNU\2024\WaterlooInternship\final_project\env\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [42]:
batch = next(iter(train_loader))
batch

DataBatch(x=[16, 11], edge_index=[2, 24], edge_attr=[24, 4], y=[4, 19], pos=[16, 3], idx=[4], name=[4], z=[16], batch=[16], ptr=[5])

In [43]:
batch.batch

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3])

In [44]:
torch.unique(data.x[:, 5], return_counts=True)

(tensor([1., 6., 7., 8., 9.]),
 tensor([1208486,  831925,  132498,  183265,    3036]))

In [61]:
torch.unique(torch.argmax(data.edge_attr, dim=-1), return_counts=True)

(tensor([0, 1, 2]), tensor([4563244,  247016,   73256]))

In [53]:
a, b = torch.unique((torch.argmax(data.edge_attr, dim=-1)+1), return_counts=True )

In [58]:
a

tensor([1, 2, 3])

In [56]:
b

tensor([4563244,  247016,   73256])

In [57]:
torch.sum(b)

tensor(4883516)

In [116]:
num_nodes = 10
atom_types = 5
supported_edges = 4
input_dim = 11
enc_dim = 100
latent_dim = 128

In [123]:

class Mol_VAE(nn.Module): 
    def __init__(self, input_dim, enc_dim, latent_dim, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        output_atoms = int(num_nodes*(atom_types +1))
        output_edges = int((num_nodes * (num_nodes-1)) * supported_edges)

        # Encoder 
        self.input_dim = input_dim 
        self.enc_dim = enc_dim
        self.enc_tConv1 = pyg_nn.TransformerConv(input_dim, enc_dim, edge_dim=supported_edges)
        self.enc_b1 = pyg_nn.BatchNorm(enc_dim)

        self.pool = Set2Set(enc_dim, 4)
        self.pool = global_max_pool

        self.enc_mu = nn.Linear(enc_dim, latent_dim)
        self.enc_logvar = nn.Linear(enc_dim, latent_dim)

        # Decoder 
        self.dec_lin_atoms = nn.Linear(latent_dim, output_atoms)
        self.dec_lin_edges = nn.Linear(latent_dim, output_edges)

    def encoder(self, x, edge_index, edge_attr, batch_index): 
        x = self.enc_tConv1(x,  edge_index, edge_attr)
        x = self.enc_b1(x)
        x = F.relu(x)
        x = self.pool(x, batch_index)

        mu, logvar = self.enc_mu(x), self.enc_logvar(x)
        return mu, logvar

    def decoder(self, z):
        atoms = self.dec_lin_atoms(z)
        edges = self.dec_lin_edges(z)
        return atoms, edges
        
    def reparameterization(self, mu, logvar): 
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(mu)
        return eps.mul(std).add_(mu)
        

    def forward(self, batch):
        # DataBatch(x=[16, 11], edge_index=[2, 24], edge_attr=[24, 4], y=[4, 19], pos=[16, 3], idx=[4], name=[4], z=[16], batch=[16], ptr=[5])
        x, edge_index, edge_attr, batch_index = batch.x, batch.edge_index, batch.edge_attr, batch.batch

        mu, logvar = self.encoder(x, edge_index, edge_attr, batch_index)
        z = self.reparameterization(mu, logvar, x)
        atoms, edges = self.decoder(z)

mol_vae = Mol_VAE(input_dim, enc_dim, latent_dim)
mol_vae(batch)


TypeError: cannot assign 'torch_geometric.nn.pool.glob.global_max_pool' as child module 'pool' (torch.nn.Module or None expected)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x11 and 15x100)

In [102]:
type(data[0])

torch_geometric.data.data.Data

In [100]:
mol_vae(data[0].x)

TypeError: forward() missing 1 required positional argument: 'edge_index'

In [89]:
a = torch.randn((10, 2))
b = torch.randn((10, 2))
a.mean(), b.mean()

(tensor(-0.2048), tensor(-0.4495))

In [90]:
a.add(b)

tensor([[-0.5987, -1.6235],
        [-1.9206, -1.9048],
        [ 0.0170, -1.3077],
        [ 0.4227,  0.1668],
        [ 0.1509, -2.8364],
        [-1.6013,  1.6137],
        [-0.4223,  1.0727],
        [ 0.5950, -2.2849],
        [-0.3501, -1.1127],
        [-2.3833,  1.2215]])

In [91]:
a.add(b)

tensor([[-0.5987, -1.6235],
        [-1.9206, -1.9048],
        [ 0.0170, -1.3077],
        [ 0.4227,  0.1668],
        [ 0.1509, -2.8364],
        [-1.6013,  1.6137],
        [-0.4223,  1.0727],
        [ 0.5950, -2.2849],
        [-0.3501, -1.1127],
        [-2.3833,  1.2215]])